In [1]:
import sqlalchemy as db
import pandas as pd
import numpy as np


path = "/home/radwa/Radwa.Usecase/staging_1/orders/orders.csv"

df_order = pd.read_csv(path)

In [2]:
df_order.columns

Index(['order_id', 'customer_id', 'order_status', 'order_date',
       'required_date', 'shipped_date', 'store_id', 'staff_id', 'Extract_time',
       'Source'],
      dtype='object')

In [3]:
from datetime import datetime
df_order['late_delivery'] = df_order['shipped_date'] > df_order['required_date']


In [4]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6460 entries, 0 to 6459
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       6460 non-null   int64 
 1   customer_id    6460 non-null   int64 
 2   order_status   6460 non-null   int64 
 3   order_date     6460 non-null   object
 4   required_date  6460 non-null   object
 5   shipped_date   6460 non-null   object
 6   store_id       6460 non-null   int64 
 7   staff_id       6460 non-null   int64 
 8   Extract_time   6460 non-null   object
 9   Source         6460 non-null   object
 10  late_delivery  6460 non-null   bool  
dtypes: bool(1), int64(5), object(5)
memory usage: 511.1+ KB


In [5]:
df_order['required_date'] = pd.to_datetime(df_order['required_date'], errors='coerce')
df_order['shipped_date'] = pd.to_datetime(df_order['shipped_date'], errors='coerce')

In [6]:
def latency_days(row):
    if pd.isna(row['shipped_date']):
        return np.nan
    latency = (row['shipped_date'] - row['required_date']).days
    return latency if latency > 0 else np.nan
df_order['latency days'] = df_order.apply(latency_days, axis=1)

In [7]:
df_order['weekend'] = df_order['shipped_date'].apply(
    lambda x: True if x is not None and x.weekday() in [0, 6] else False
)

In [8]:
df_order

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,Extract_time,Source,late_delivery,latency days,weekend
0,1,259,4,2016-01-01 00:00:00.000000000,2016-01-03,2016-01-03,1,2,2024-05-02 11:54:57.869750,postgres,False,NaN,True
1,2,1212,4,2016-01-01 00:00:00.000000000,2016-01-04,2016-01-03,2,6,2024-05-02 11:54:57.869750,postgres,False,NaN,True
2,3,523,4,2016-01-02 00:00:00.000000000,2016-01-05,2016-01-03,2,7,2024-05-02 11:54:57.869750,postgres,False,NaN,True
3,4,175,4,2016-01-03 00:00:00.000000000,2016-01-04,2016-01-05,1,3,2024-05-02 11:54:57.869750,postgres,True,1.0,False
4,5,1324,4,2016-01-03 00:00:00.000000000,2016-01-06,2016-01-06,2,6,2024-05-02 11:54:57.869750,postgres,False,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6455,1611,6,3,2018-09-06 00:00:00.000000000,2018-09-06,2018-04-02,2,7,2024-05-02 11:54:57.869750,postgres,False,NaN,True
6456,1612,3,3,2018-10-21 00:00:00.000000000,2018-10-21,2018-04-02,1,3,2024-05-02 11:54:57.869750,postgres,False,NaN,True
6457,1613,1,3,2018-11-18 00:00:00.000000000,2018-11-18,2018-04-02,2,6,2024-05-02 11:54:57.869750,postgres,False,NaN,True
6458,1614,135,3,2018-11-28 00:00:00.000000000,2018-11-28,2018-04-02,3,8,2024-05-02 11:54:57.869750,postgres,False,NaN,True


In [9]:
import os

csv_path = "/home/radwa/Radwa.Usecase/staging_2/orders/orders.csv" 

directory = os.path.dirname(csv_path)
if not os.path.exists(directory):
    os.makedirs(directory)
    
df_order.to_csv(csv_path, index=False)